In [1]:
'''
The programm aims to predict the weather
'''
import xgboost as xgb
import pandas as pd
import numpy as np
#import datetime
import pickle
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
from multiprocessing import Pool
import sys
from sklearn.model_selection import GridSearchCV
import warnings
import sklearn
import random
from tqdm import tqdm
import lightgbm as lgb


# 特征汇总 
# 十个风速模型 m1..m10
# 未来一小时十个风速模型 m1_..m10_
# 十个风速方差，均值，最大值，最小值
# 小时 h

# 对未来天气进行初步预测 ####################################
def feature_selection_x(param):
    (input_weather,day) = param
    x_max = 548
    y_max = 421
    train_x = np.zeros((4152744, 129),dtype=np.float16)
    i = 0
    for h in tqdm(range(3,21)):
        for x in range(x_max):
            for y in range(y_max):
                temp = []
                # now weather
                temp_1 = []
                temp_1_wind = []
                for mod in range(1, 11):
                    temp_1_wind.append(input_weather[str(mod)][str(day)][str(h)][0,x,y])
                
                mean = np.mean(temp_1_wind)
                var = np.var(temp_1_wind)
                max_wind = np.max(temp_1_wind)
                min_wind = np.min(temp_1_wind)
                basic_wind_1 = [mean,var,max_wind,min_wind]
                #temp_1_wind.extend([mean,var,max_wind,min_wind])

                temp_1_rainfall = []
                for mod in range(1, 11):
                    temp_1_rainfall.append(input_weather[str(mod)][str(day)][str(h)][1,x,y])
                mean = np.mean(temp_1_rainfall)
                var = np.var(temp_1_rainfall)
                max_wind = np.max(temp_1_rainfall)
                min_wind = np.min(temp_1_rainfall)
                basic_rainfall_1 = [mean,var,max_wind,min_wind]

                temp_1.extend(temp_1_wind)
                temp_1.extend(basic_wind_1)
                temp_1.extend(temp_1_rainfall)
                temp_1.extend(basic_rainfall_1)
                # next hour weather
                temp_2 = []
                temp_2_wind = []
                for mod in range(1, 11):
                    try:
                        temp_2_wind.append(input_weather[str(mod)][str(day)][str(h+1)][0,x,y])
                    except:
                        temp_2_wind.append(input_weather[str(mod)][str(day)][str(h)][0,x,y])
                mean = np.mean(temp_2_wind)
                var = np.var(temp_2_wind)
                max_wind = np.max(temp_2_wind)
                min_wind = np.min(temp_2_wind)
                basic_wind_2 = [mean,var,max_wind,min_wind]

                #temp_2_wind.extend([mean,var,max_wind,min_wind])
                
                temp_2_rainfall = []
                for mod in range(1, 11):
                    try:
                        temp_2_rainfall.append(input_weather[str(mod)][str(day)][str(h+1)][1,x,y])
                    except:
                        temp_2_rainfall.append(input_weather[str(mod)][str(day)][str(h)][1,x,y])
                mean = np.mean(temp_2_rainfall)
                var = np.var(temp_2_rainfall)
                max_wind = np.max(temp_2_rainfall)
                min_wind = np.min(temp_2_rainfall)
                basic_rainfall_2 = [mean,var,max_wind,min_wind]

                #temp_2_rainfall.extend([mean,var,max_wind,min_wind])

                temp_2.extend(temp_2_wind)
                temp_2.extend(basic_wind_2)
                temp_2.extend(temp_2_rainfall)
                temp_2.extend(basic_rainfall_2)

                # last hour weather
                temp_3 = []
                temp_3_wind = []
                for mod in range(1, 11):
                    try:
                        temp_3_wind.append(input_weather[str(mod)][str(day)][str(h-1)][0,x,y])
                    except:
                        temp_3_wind.append(input_weather[str(mod)][str(day)][str(h)][0,x,y])
                mean = np.mean(temp_3_wind)
                var = np.var(temp_3_wind)
                max_wind = np.max(temp_3_wind)
                min_wind = np.min(temp_3_wind)
                basic_wind_3 = [mean,var,max_wind,min_wind]

                temp_3_rainfall = []
                for mod in range(1, 11):
                    try:
                        temp_3_rainfall.append(input_weather[str(mod)][str(day)][str(h-1)][1,x,y])
                    except:
                        temp_3_rainfall.append(input_weather[str(mod)][str(day)][str(h)][1,x,y])
                mean = np.mean(temp_3_rainfall)
                var = np.var(temp_3_rainfall)
                max_wind = np.max(temp_3_rainfall)
                min_wind = np.min(temp_3_rainfall)
                basic_rainfall_3 = [mean,var,max_wind,min_wind]

                temp_3.extend(temp_3_wind)
                temp_3.extend(basic_wind_3)
                temp_3.extend(temp_3_rainfall)
                temp_3.extend(basic_rainfall_3)
                
                # add info
                average = [basic_rainfall_2[0]-basic_rainfall_1[0],basic_rainfall_1[0]-basic_rainfall_3[0],
                          basic_wind_2[0]-basic_wind_1[0],basic_wind_1[0]-basic_wind_3[0]]
                add_model_1 = list(np.array(temp_2_wind)-np.array(temp_1_wind))
                add_model_2 = list(np.array(temp_1_wind)-np.array(temp_3_wind))
                add_model_3 = list(np.array(temp_2_rainfall)-np.array(temp_1_rainfall))
                add_model_4 = list(np.array(temp_1_rainfall)-np.array(temp_3_rainfall))
                
                temp.extend(temp_1)
                temp.extend(temp_2)
                temp.extend(temp_3)
                temp.extend(average)
                temp.extend(add_model_1)
                temp.extend(add_model_2)
                temp.extend(add_model_3)
                temp.extend(add_model_4)
                # r=5 region 平均风速增长/安全面积
                # r=10 region 平均风速增长/安全面积 
                # r=20 region 平均风速增长/安全面积
                # 最近的风险点的距离
                # 风险点增长
                # 
                temp.append(h)
                #print(len(temp))
                train_x[i,:] = temp
                i += 1

    # return (day,train_x)
    return [day,train_x]


def feature_selection_y(output_weather):
    x_max = 548
    y_max = 421
    wind_y = []
    rainfall_y = []
    prob_y = []
    for day in range(1,6):
        for h in tqdm(range(3,21)):
            for x in range(x_max):
                for y in range(y_max):
                    wind_y.append(output_weather[str(day)][str(h)][0,x,y])
                    rainfall_y.append(output_weather[str(day)][str(h)][1,x,y])
                    if output_weather[str(day)][str(h)][0,x,y]<15 and output_weather[str(day)][str(h)][1,x,y]<4:
                        prob_y.append(1)
                    else:
                        prob_y.append(0)
    return wind_y,rainfall_y,prob_y
# 特征汇总 
# 十个风速模型 m1..m10
# 未来一小时十个风速模型 m1_..m10_
# 十个风速方差，均值，最大值，最小值
# 小时 h


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
a = np.array([1,2])
b = np.array([2,3])
b-a

array([1, 1])

In [13]:
# data source
input_weather = open('weather_train_1_5.pkl', 'rb')
input_weather = pickle.load(input_weather)


In [ ]:
# multi proc to generate train data
pool = Pool(3)
params = [(input_weather,i) for i in range(1,6)]
result = pool.map(feature_selection_x, params)
pool.close()  # 关闭进程池，不再接受新的进程
pool.join()  # 主进程阻塞等待子进程的退出

In [8]:
result = sorted(result,key=lambda x:x[0])

In [10]:
weather = np.concatenate((result[0][1],result[1][1],result[2][1],result[3][1],result[4][1]),axis=0)

In [12]:
np.save("layer_1_basicinfo_129.npy", weather)

In [6]:
output_weather = open('weather_train_label_1_5.pkl', 'rb')
output_weather = pickle.load(output_weather)
wind_y,rainfall_y,prob_y = feature_selection_y(output_weather)
#del wind_y,rainfall_y


100%|██████████| 18/18 [00:18<00:00,  1.00s/it]


In [7]:
train_x = np.load("layer_1_basicinfo_129.npy")

In [3]:
# 用平均值预测
sample_index = np.load('sample_index.npy')
x_sample = np.load('./feature/layer_2_X_129.npy')
#y_sample = np.load('./feature/layer_2_label.npy')


In [8]:
y_sample = np.array(wind_y)

In [9]:
x_sample =train_x
del train_x

In [4]:
sample_index = np.load('sample_index.npy')
x_sample = train_x[sample_index]
y_sample = np.array(prob_y)[sample_index]
del train_x

In [17]:
# 回归模型，预测基本数据
param = {
    'num_leaves':225, 
    #'num_leaves':100, 
    'num_trees':150, 
    'objective':'regression',
    'metric':'l2',
    'learning_rate': 0.05,
    'num_threads':8,
    #'min_sum_hessian_in_leaf':100
}
#x_sample = train_x
#y_sample = np.array(wind_y)
kfolds = 12
kf = KFold(n_splits=kfolds,shuffle=True,random_state=2016)
result = np.zeros((len(prob_y),))
i=0
num_round = 130
for train_index,test_index in kf.split(x_sample):
    X_train, X_test = x_sample[train_index], x_sample[test_index]
    y_train, y_test = y_sample[train_index], y_sample[test_index]
    train_data = lgb.Dataset(X_train,label=y_train)
    #validation_data = lgb.Dataset(X_test,label=y_test)
    bst = lgb.train(param, train_data,num_round)
    result[test_index] = bst.predict(X_test)
    print('测试误差',mse(y_test,result[test_index]))
    #nonsample_result[i,:] = bst.predict(x_nonsample)
    file_name = 'model_1_wind_shuffle_'+str(i)+'.model'
    bst.save_model(file_name)
    i+=1

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


测试误差 0.610817918183
测试误差 0.628651945814
测试误差 0.627866566635
测试误差 0.629099255005
测试误差 0.62665532363
测试误差 0.630623333217
测试误差 0.631695952863
测试误差 0.626688777242
测试误差 0.629042242385
测试误差 0.629708087361
测试误差 0.630307579244
测试误差 0.629629798341


In [18]:
# 保存风速数据到 pickle
i = 0
x_max = 548
y_max = 421
weather = {}
for day in range(1,6):
    weather_h = {}
    for h in range(3,21):
        data = np.zeros((x_max,y_max),np.float16)
        for x in range(x_max):
            for y in range(y_max):
                data[x,y] = result[i]
                i += 1
        weather_h[str(h)] = data
    weather[str(day)] = weather_h
import pickle
f1 = open('layer_1_wind_shuffle.pkl', 'wb')
pickle.dump(weather,f1)
f1.close()

In [3]:
# 平均值模型直接预测概率，不经过模型融合,用于寻找参数
param = {
    'num_leaves':200, 
    'num_trees':300, 
    'objective':'binary',
    'metric':'auc',
    'learning_rate': 0.1,
    'num_threads':10,
    'min_sum_hessian_in_leaf':100
}

kfolds = 10
kf = KFold(n_splits=kfolds,shuffle=False,random_state=2016)
sample_result = np.zeros((len(y_sample),))
i=0
num_round = 500
for train_index,test_index in kf.split(x_sample):

    X_train, X_test = x_sample[train_index], x_sample[test_index]
    y_train, y_test = y_sample[train_index], y_sample[test_index]
    train_data = lgb.Dataset(X_train,label=y_train)
    validation_data = lgb.Dataset(X_test,label=y_test)
    #test_data = lgb.Dataset(x_test, label=test_label, reference=train_data)
    bst = lgb.train( param,train_data, valid_sets=[validation_data],early_stopping_rounds=100)
    break
    #bst = lgb.train(param, train_data, num_round)
    sample_result[test_index] = bst.predict(X_test)
    print('测试误差',mse(y_test,sample_result[test_index]))
    nonsample_result[i,:] = bst.predict(x_nonsample)
    i+=1

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.983407
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.984095
[3]	valid_0's auc: 0.984567
[4]	valid_0's auc: 0.984692
[5]	valid_0's auc: 0.984895
[6]	valid_0's auc: 0.985057
[7]	valid_0's auc: 0.985122
[8]	valid_0's auc: 0.985158
[9]	valid_0's auc: 0.985191
[10]	valid_0's auc: 0.985237
[11]	valid_0's auc: 0.985307
[12]	valid_0's auc: 0.98534
[13]	valid_0's auc: 0.985395
[14]	valid_0's auc: 0.985407
[15]	valid_0's auc: 0.985464
[16]	valid_0's auc: 0.985495
[17]	valid_0's auc: 0.985502
[18]	valid_0's auc: 0.985523
[19]	valid_0's auc: 0.985553
[20]	valid_0's auc: 0.985569
[21]	valid_0's auc: 0.985607
[22]	valid_0's auc: 0.985603
[23]	valid_0's auc: 0.985609
[24]	valid_0's auc: 0.985624
[25]	valid_0's auc: 0.985634
[26]	valid_0's auc: 0.985651
[27]	valid_0's auc: 0.985675
[28]	valid_0's auc: 0.985716
[29]	valid_0's auc: 0.985759
[30]	valid_0's auc: 0.985785
[31]	valid_0's auc: 0.985787
[32]	valid_0's auc: 0.985801
[33]	valid_0's auc:

KeyboardInterrupt: 

In [6]:
# 直接预测概率，不经过模型融合,用于寻找参数
param = {
    'num_leaves':200, 
    'num_trees':300, 
    'objective':'binary',
    'metric':'auc',
    'learning_rate': 0.1,
    'num_threads':10,
    'min_sum_hessian_in_leaf':100
}

kfolds = 10
kf = KFold(n_splits=kfolds,shuffle=False,random_state=2016)
sample_result = np.zeros((len(y_sample),))
i=0
num_round = 500
for train_index,test_index in kf.split(x_sample):

    X_train, X_test = x_sample[train_index], x_sample[test_index]
    y_train, y_test = y_sample[train_index], y_sample[test_index]
    train_data = lgb.Dataset(X_train,label=y_train)
    validation_data = lgb.Dataset(X_test,label=y_test)
    #test_data = lgb.Dataset(x_test, label=test_label, reference=train_data)
    bst = lgb.train( param,train_data, valid_sets=[validation_data],early_stopping_rounds=100)
    break
    #bst = lgb.train(param, train_data, num_round)
    sample_result[test_index] = bst.predict(X_test)
    print('测试误差',mse(y_test,sample_result[test_index]))
    nonsample_result[i,:] = bst.predict(x_nonsample)
    i+=1

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.983548
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.98417
[3]	valid_0's auc: 0.984572
[4]	valid_0's auc: 0.98473
[5]	valid_0's auc: 0.984839
[6]	valid_0's auc: 0.984903
[7]	valid_0's auc: 0.985061
[8]	valid_0's auc: 0.985185
[9]	valid_0's auc: 0.985249
[10]	valid_0's auc: 0.985306
[11]	valid_0's auc: 0.985339
[12]	valid_0's auc: 0.985388
[13]	valid_0's auc: 0.985429
[14]	valid_0's auc: 0.985473
[15]	valid_0's auc: 0.98551
[16]	valid_0's auc: 0.985535
[17]	valid_0's auc: 0.985549
[18]	valid_0's auc: 0.985573
[19]	valid_0's auc: 0.985559
[20]	valid_0's auc: 0.985578
[21]	valid_0's auc: 0.98559
[22]	valid_0's auc: 0.985569
[23]	valid_0's auc: 0.985595
[24]	valid_0's auc: 0.985581
[25]	valid_0's auc: 0.985602
[26]	valid_0's auc: 0.985613
[27]	valid_0's auc: 0.985628
[28]	valid_0's auc: 0.985675
[29]	valid_0's auc: 0.985713
[30]	valid_0's auc: 0.985733
[31]	valid_0's auc: 0.98573
[32]	valid_0's auc: 0.985765
[33]	valid_0's auc: 0.9

KeyboardInterrupt: 

In [11]:
# 训练模型
# 
param = {
    'num_leaves':200, 
    'num_trees':250, 
    'objective':'binary',
    'metric':'auc',
    'learning_rate': 0.1,
    'num_threads':10,
    'min_sum_hessian_in_leaf':100
}
train_data = lgb.Dataset(x_sample,label=y_sample)
bst = lgb.train(param,train_data)
bst.save_model('layer_1_model_1_prob.txt')

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [8]:
# 训练模型
# 
param = {
    'num_leaves':200, 
    'num_trees':250, 
    'objective':'binary',
    'metric':'auc',
    'learning_rate': 0.1,
    'num_threads':10,
    'min_sum_hessian_in_leaf':100
}
train_data = lgb.Dataset(x_sample,label=y_sample)
bst = lgb.train(param,train_data)
bst.save_model('layer_1_model_2_ave_prob.txt')

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


KeyboardInterrupt: 

In [ ]:
bst = lgb.Booster(model_file='layer_1_model_2_ave_prob.txt') 
predict_x = np.load('X_test_ave.npy')
predict_prob = bst.predict(predict_x)

In [12]:
# 预测输出，初步预测输出
predict_x = np.load('layer_3_basicinfo.npy')
predict_prob = bst.predict(predict_x)

In [ ]:
# 保存数据到pickle
i = 0
x_max = 548
y_max = 421
weather = {}
for day in range(1,6):
    weather_h = {}
    for h in range(3,21):
        data = np.zeros((x_max,y_max),np.float16)
        for x in range(x_max):
            for y in range(y_max):
                data[x,y] = predict_prob[i]
                i += 1
        weather_h[str(h)] = data
    weather[str(day)] = weather_h
import pickle
f1 = open('layer_1_output_prob_ave.pkl', 'wb')
pickle.dump(weather,f1)
f1.close()

In [1]:

    
    #bst = lgb.train(param,train_data,valid_sets=validation_data,early_stopping_rounds=50)
    # best result 3.307
    #break
    #xg_train = xgb.DMatrix(X_train, label = y_train)  
    #xgeval = xgb.DMatrix(X_test,label=y_test)
    #watchlist = [(xg_train, 'train'), (xgeval, 'val')]
    #bst2 = xgb.train(param, xg_train, num_round,watchlist, early_stopping_rounds = 100)
    # best result smaller than 2.2 in 62 rounds
    #break
    # mse
    #print('训练误差',mse(y_train,bst.predict(xg_train)))
    #predict_x = xgb.DMatrix(np.array(X_test))


In [11]:
print(nonsample_result[:,1:10])
print(sample_result[0:10])

[[ 12.8381299   12.67829644  12.49357571  12.62091972  13.11059302
   15.4390915   15.41172877  15.35912885  13.04983685]
 [ 12.79528249  12.68653863  12.40559759  12.69644404  13.05902716
   15.18816091  15.1470643   15.13895261  13.05516924]
 [ 12.77775689  12.66773553  12.48897243  12.63885546  13.555559
   15.40350466  15.33537203  15.26145667  13.06803193]
 [ 12.84612792  12.62040606  12.07626445  12.46056851  13.323624
   15.00568061  14.9951347   14.99411874  13.18976548]
 [ 12.93796562  12.79704992  12.74291576  12.65589549  13.18260743
   14.90393838  14.927097    15.01066666  13.00541304]
 [ 12.98128649  12.72400224  12.3574961   12.8301991   13.14539873
   15.07529364  15.05802388  15.00805135  13.24577298]
 [ 12.86725314  12.78413363  12.3315796   12.53764423  13.10881684
   15.03923253  15.00455467  14.8898428   13.2399174 ]
 [ 12.8728845   12.7088336   12.53070187  12.89741614  13.07289398
   15.15279661  15.11160781  15.0317126   13.13704855]
 [ 12.65139305  12.4789665  

In [12]:
nonsample_wind = nonsample_result.mean(axis=0)
i = 0
for item in index_nonsample:
    train_y[item] = nonsample_wind[i]
    i += 1
j = 0
for item in index_sample:
    train_y[item] = sample_result[j]
    j += 1

In [13]:
np.save("layer_1_gbm_output.npy", train_y)

In [14]:
output_weather = open('weather_output.pkl', 'rb')
output_weather = pickle.load(output_weather)
i = 0
x_max = 548
y_max = 421
for day in range(1,6):
    print(day)
    for h in range(3,21):
        for x in range(x_max):
            for y in range(y_max):
                output_weather[str(day)][str(h)][x][y] = np.float16(train_y[i])
                i += 1
import pickle
f1 = open('layer_2_gbm_input.pkl', 'wb')
pickle.dump(output_weather,f1)
f1.close()

1
2
3
4
5


In [ ]:
sample_result[test_index][1000:2000] 

In [16]:
y_test[1000:2000]

array([ 30.6 ,  31.3 ,  31.9 ,  32.3 ,  32.6 ,  32.5 ,  32.2 ,  31.7 ,
        31.8 ,  32.  ,  32.6 ,  32.8 ,  32.7 ,  32.8 ,  32.6 ,  32.3 ,
        31.4 ,  30.7 ,  30.6 ,  30.9 ,  31.2 ,   3.12,   3.02,   2.53,
         1.15,   2.12,   4.1 ,   3.25,   1.27,   1.45,   0.7 ,   0.45,
         0.27,   0.87,   1.07,   1.9 ,   1.15,   1.06,   1.42,   1.73,
         1.52,   2.37,   2.52,   2.67,   2.46,   2.87,   4.52,   4.8 ,
         6.4 ,   6.56,   7.16,   7.2 ,   7.7 ,   7.73,   8.4 ,   8.77,
         9.11,   9.33,   9.39,  11.4 ,  10.2 ,   9.85,  10.6 ,  10.8 ,
        10.8 ,  10.8 ,  11.  ,  10.9 ,  11.4 ,  12.8 ,  13.1 ,  13.1 ,
        13.1 ,  13.4 ,  14.  ,  13.8 ,  13.8 ,  13.8 ,  14.3 ,  14.6 ,
        14.7 ,  14.7 ,  14.9 ,  15.4 ,  16.2 ,  17.  ,  17.4 ,  17.6 ,
        17.4 ,  17.  ,  16.5 ,  16.4 ,  16.6 ,  16.7 ,  16.6 ,  16.7 ,
        16.1 ,  15.6 ,  15.8 ,  15.8 ,  14.9 ,  14.3 ,  14.  ,  13.8 ,
        14.2 ,  15.6 ,  17.2 ,  18.3 ,  18.6 ,  18.3 ,  18.  ,  17.9 ,
      

In [105]:
print('测试误差',mse(y_test,bst.predict(X_test)))

测试误差 4.361824894


In [59]:
input_weather_ave = open('weather_average_input.pkl', 'rb')
input_weather_ave = pickle.load(input_weather_ave)
input_y_ave = feature_selection_y(input_weather_ave)

100%|██████████| 18/18 [00:02<00:00,  6.73it/s]


In [60]:
print('平均值法误差',mse(train_y,input_y_ave))

平均值法误差 4.58640380251


In [ ]:
bst = lgb.train( param,train_data, valid_sets=[test_data],early_stopping_rounds=100)